## Data Preparation for question 2

2. What are the factors most strongly associated with restaurants being closed?
How accurately can you predict when a restaurant in the dataset will be closed?

In [58]:
#imports
import numpy as np
import pandas as pd
import ast

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime

### Get initial dataframes

In [22]:
business = pd.read_csv("data/business.csv")
print(len(business))
print(business.columns)
business.head(2)

7148
Index(['address', 'AcceptsInsurance', 'AgesAllowed', 'Alcohol',
       'Ambience.casual', 'Ambience.classy', 'Ambience.hipster',
       'Ambience.intimate', 'Ambience.romantic', 'Ambience.touristy',
       'Ambience.trendy', 'Ambience.upscale', 'BestNights.friday',
       'BestNights.monday', 'BestNights.saturday', 'BestNights.sunday',
       'BestNights.thursday', 'BestNights.tuesday', 'BestNights.wednesday',
       'BikeParking', 'BusinessAcceptsCreditCards', 'BusinessParking.garage',
       'BusinessParking.lot', 'BusinessParking.street',
       'BusinessParking.valet', 'BusinessParking.validated',
       'ByAppointmentOnly', 'Caters', 'CoatCheck',
       'DietaryRestrictions.dairy-free', 'DietaryRestrictions.gluten-free',
       'DietaryRestrictions.halal', 'DietaryRestrictions.kosher',
       'DietaryRestrictions.soy-free', 'DietaryRestrictions.vegan',
       'DietaryRestrictions.vegetarian', 'DogsAllowed', 'DriveThru',
       'GoodForDancing', 'GoodForKids', 'GoodForMeal.bre

,address,AcceptsInsurance,AgesAllowed,Alcohol,Ambience.casual,Ambience.classy,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,...,hours.Wednesday,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,2459 Yonge St,NaN,NaN,full_bar,False,False,False,False,True,False,...,9:00-22:00,0,43.71140,-79.39934,Alize Catering,Yonge and Eglinton,M4P 2H6,12,3.0,ON
1,1058 Gerrard Street E,NaN,NaN,full_bar,True,False,False,False,False,False,...,11:30-2:00,1,43.66926,-79.33590,Chula Taberna Mexicana,Leslieville,M4M 3A6,39,3.5,ON


In [23]:
business[business['review_count'].isnull()]

,address,AcceptsInsurance,AgesAllowed,Alcohol,Ambience.casual,Ambience.classy,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,...,hours.Wednesday,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state


In [24]:
for b in business['categories']:
    if b == "[]":
        print(b)

In [43]:
print(len(business['Smoking']))
sum(business['Smoking'].isnull())

7148


6477

In [44]:
print(len(business['AgesAllowed']))
sum(business['AgesAllowed'].isnull())

7148


7141

In [45]:
print(len(business['Alcohol']))
sum(business['Alcohol'].isnull())

7148


1614

In [25]:
reviews = pd.read_csv("data/reviews.csv")
print(len(reviews))
reviews.head(2)

303780


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,9_CGhHMz8698M9-PkVf0CQ,2,2012-05-11,0,ymAUG8DZfQcFTBSOiaNN4w,4,Who would have guess that you would be able to...,0,u0LXt3Uea_GidxRW1xcsfg
1,5r6-G9C4YLbC7Ziz57l3rQ,0,2013-02-09,0,w41ZS9shepfO3uEyhXEWuQ,3,"Not bad!! Love that there is a gluten-free, ve...",1,u0LXt3Uea_GidxRW1xcsfg


### Business dataset
<br>
Used the following variables from the business initial dataset: 
<ul>
    <li> latitude
        <li> longitude
            <li> review_count
                <li> stars
    <ul>
These varibales can be taken directly from the large dataset without any transformation 
        <br><br>
The categories variable could also provide some insight into whether a a resturnat is closed or not. For example because we might expect some resturant categories are less popular leading to their closure. Each resturant has a list of categories associated to it. For example, ['Italian', 'French', 'Restaurants']. Hence, it is nessary to transfore this categoires variable. This could be done using indicator variables (i.e. x_cat =1 if category is in list of resturants categories and 0 otherwise). The promblem is that there are 323 unique categories and this would result in 323 variables. To reduce the complexity of the model it is nessary to select on the most predictive variables. This was done by find the top 10 most common categories for both resuranct open and closed. This resulted in 2 lists. We then found the categories that where not in both lists. These resturanct are potentially popular for open resturants and not popular for closed resuturants and visa versa. In other words, they will allow us to distinguish between the two types of resturants. 
<br>
        Italian and Japanese in the top 10 list for closed and not open. Pizza, Coffee & Tea in open but not closed 
<br><br>
These variables had no missing values. The rest of the variables from the business dataset where quite sparce. FOr instance, 1614/7148 of Alcohol was missing, 6477/7148 of smoking variable was missing and only 7 resturants gave information on their AgesAllowed variable. Hence, the rest of the variables from the business dataset where ignored. More potential explanaotry variables where obtained from the review dataset:


 

In [26]:
question2_1 = business[['business_id','name','latitude','longitude','review_count','stars',
                          'is_open']]
question2_1.head()

,business_id,name,latitude,longitude,review_count,stars,is_open
0,l09JfMeQ6ynYs5MCJtrcmQ,Alize Catering,43.71140,-79.39934,12,3.0,0
1,1K4qrnfyzKzGgJPBEcJaNQ,Chula Taberna Mexicana,43.66926,-79.33590,39,3.5,1
2,nbhBRhZtdaZmMMeb2i02pg,Sunnyside Grill,43.78182,-79.49043,3,5.0,1
3,FXHfcFVEfI1vVngW2gVOpw,Bampot House of Tea & Board Games,43.66158,-79.40888,55,4.0,1
4,VXH7zXcZzXlmAVN8GSjGRQ,Thai Express,43.77488,-79.49462,5,3.0,1


### Categories 

In [27]:
list(business['categories'])[0]

"['Italian', 'French', 'Restaurants']"

In [28]:
categories = list(business['categories'])
categories_list =  [item for sublist in categories for item in ast.literal_eval(sublist)]
print(len(set(categories_list)))
popular_categories = pd.Series(categories_list).value_counts()[1:21]
popular_categories

323


Food                      1478
Nightlife                  940
Bars                       907
Canadian (New)             629
Sandwiches                 619
Breakfast & Brunch         606
Italian                    566
Chinese                    559
Cafes                      551
Pizza                      509
Coffee & Tea               496
Japanese                   490
Fast Food                  424
Burgers                    417
Sushi Bars                 341
American (Traditional)     325
Indian                     291
Middle Eastern             275
Thai                       275
Mediterranean              266
dtype: int64

In [29]:
categories = list(business[business['is_open'] == 0]['categories'])
categories_list =  [item for sublist in categories for item in ast.literal_eval(sublist)]
popular_categories_1 = pd.Series(categories_list).value_counts()[1:11].index
popular_categories_1

Index(['Food', 'Nightlife', 'Bars', 'Canadian (New)', 'Italian',
       'Breakfast & Brunch', 'Japanese', 'Chinese', 'Sandwiches', 'Cafes'],
      dtype='object')

In [30]:
categories = list(business[business['is_open'] == 1]['categories'])
categories_list =  [item for sublist in categories for item in ast.literal_eval(sublist)]
popular_categories_2 = pd.Series(categories_list).value_counts()[1:11].index
popular_categories_2

Index(['Food', 'Nightlife', 'Bars', 'Sandwiches', 'Breakfast & Brunch',
       'Canadian (New)', 'Cafes', 'Chinese', 'Pizza', 'Coffee & Tea'],
      dtype='object')

In [31]:
uncommon = []
for p in popular_categories_1: 
    if p not in popular_categories_2:
        print(p)
        uncommon.append(p)
print()
for p in popular_categories_2: 
    if p not in popular_categories_1:
        print(p)
        uncommon.append(p) 
uncommon

Italian
Japanese

Pizza
Coffee & Tea


['Italian', 'Japanese', 'Pizza', 'Coffee & Tea']

In [32]:
def getIndicator(name):
    """
    Creates an idicator variable for whether a resturant has a specific category
    name: category name
    """
    indicator = []
    for c in list(business['categories']):
        i = 0
        if name in c:
            i = 1
        indicator.append(i)
    return(indicator)

In [33]:
for c in uncommon:
    question2_1[c] = getIndicator(c)
question2_1.head()

/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,business_id,name,latitude,longitude,review_count,stars,is_open,Italian,Japanese,Pizza,Coffee & Tea
0,l09JfMeQ6ynYs5MCJtrcmQ,Alize Catering,43.71140,-79.39934,12,3.0,0,1,0,0,0
1,1K4qrnfyzKzGgJPBEcJaNQ,Chula Taberna Mexicana,43.66926,-79.33590,39,3.5,1,0,0,0,0
2,nbhBRhZtdaZmMMeb2i02pg,Sunnyside Grill,43.78182,-79.49043,3,5.0,1,0,0,0,0
3,FXHfcFVEfI1vVngW2gVOpw,Bampot House of Tea & Board Games,43.66158,-79.40888,55,4.0,1,0,0,0,1
4,VXH7zXcZzXlmAVN8GSjGRQ,Thai Express,43.77488,-79.49462,5,3.0,1,0,0,0,0


### Hours

In [19]:
hours = business[['business_id','hours.Friday', 'hours.Monday',
       'hours.Saturday', 'hours.Sunday', 'hours.Thursday', 'hours.Tuesday',
       'hours.Wednesday']]
hours.head()
# Too sparce

,business_id,hours.Friday,hours.Monday,hours.Saturday,hours.Sunday,hours.Thursday,hours.Tuesday,hours.Wednesday
0,l09JfMeQ6ynYs5MCJtrcmQ,9:00-22:00,9:00-22:00,9:00-22:00,9:00-22:00,9:00-22:00,9:00-22:00,9:00-22:00
1,1K4qrnfyzKzGgJPBEcJaNQ,11:30-2:00,11:30-2:00,10:30-2:00,10:30-2:00,11:30-2:00,11:30-2:00,11:30-2:00
2,nbhBRhZtdaZmMMeb2i02pg,7:00-15:00,7:00-15:00,7:00-15:00,7:00-15:00,7:00-15:00,7:00-15:00,7:00-15:00
3,FXHfcFVEfI1vVngW2gVOpw,12:00-1:30,17:00-23:00,12:00-1:30,12:00-23:00,11:00-23:00,11:00-23:00,11:00-23:00
4,VXH7zXcZzXlmAVN8GSjGRQ,10:00-23:00,10:00-23:00,11:00-22:00,11:00-22:00,10:00-23:00,10:00-23:00,10:00-23:00


## review datset

<ul>
    <li> Number 5,4,3,2 and 1 star ratings 
    <li> Average length of text reviews
    <li> Year of last review
</ul>

### Stars 

In [34]:
stars = reviews[["business_id","stars"]]
print(len(stars))
stars.head()

303780


,business_id,stars
0,9_CGhHMz8698M9-PkVf0CQ,4
1,5r6-G9C4YLbC7Ziz57l3rQ,3
2,z8oIoCT1cXz7gZP5GeU5OA,4
3,XWTPNfskXoUL-Lf32wSk0Q,3
4,RtUvSWO_UZ8V3Wpj0n077w,3


In [35]:
for star_value in range(1,6):
    star_index = [1 if x==star_value else 0 for x in stars['stars']]
    stars['stars_{}'.format(star_value)] = star_index
stars_count = stars.groupby("business_id",as_index=False).sum()
stars_count.drop(labels=['stars'],axis=1,inplace=True)
stars_count.head()

/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,business_id,stars_1,stars_2,stars_3,stars_4,stars_5
0,--DaPTJW3-tB1vP-PfdTEg,4,2,8,14,11
1,--SrzpvFLwP_YFwB_Cetow,3,6,9,17,9
2,-0NhdsDJsdarxyDPR523ZQ,4,2,9,6,1
3,-0RRiWDtfnS16AKCtfvBZg,0,0,1,0,4
4,-0aOudcaAyac0VJbMX-L1g,4,2,1,2,4


In [36]:
print(len(stars[stars['stars']==1]),sum(stars_count['stars_1']))

28891 28891


In [37]:
question2_1 = pd.merge(question2_1,stars_count)
question2_1.head()

,business_id,name,latitude,longitude,review_count,stars,is_open,Italian,Japanese,Pizza,Coffee & Tea,stars_1,stars_2,stars_3,stars_4,stars_5
0,l09JfMeQ6ynYs5MCJtrcmQ,Alize Catering,43.71140,-79.39934,12,3.0,0,1,0,0,0,2,2,4,3,1
1,1K4qrnfyzKzGgJPBEcJaNQ,Chula Taberna Mexicana,43.66926,-79.33590,39,3.5,1,0,0,0,0,0,4,11,20,4
2,nbhBRhZtdaZmMMeb2i02pg,Sunnyside Grill,43.78182,-79.49043,3,5.0,1,0,0,0,0,0,0,0,0,3
3,FXHfcFVEfI1vVngW2gVOpw,Bampot House of Tea & Board Games,43.66158,-79.40888,55,4.0,1,0,0,0,1,4,3,4,14,30
4,VXH7zXcZzXlmAVN8GSjGRQ,Thai Express,43.77488,-79.49462,5,3.0,1,0,0,0,0,1,0,2,1,1


# Text

In [53]:
text = reviews[["business_id","text"]]
text['length'] = [len(x) for x in text['text']]
text= text.groupby(["business_id"],as_index=False).mean()

question2_1 = pd.merge(question2_1,text)
question2_1.head()

/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,business_id,name,latitude,longitude,review_count,stars,is_open,Italian,Japanese,Pizza,Coffee & Tea,stars_1,stars_2,stars_3,stars_4,stars_5,length
0,l09JfMeQ6ynYs5MCJtrcmQ,Alize Catering,43.71140,-79.39934,12,3.0,0,1,0,0,0,2,2,4,3,1,1475.750000
1,1K4qrnfyzKzGgJPBEcJaNQ,Chula Taberna Mexicana,43.66926,-79.33590,39,3.5,1,0,0,0,0,0,4,11,20,4,643.153846
2,nbhBRhZtdaZmMMeb2i02pg,Sunnyside Grill,43.78182,-79.49043,3,5.0,1,0,0,0,0,0,0,0,0,3,218.333333
3,FXHfcFVEfI1vVngW2gVOpw,Bampot House of Tea & Board Games,43.66158,-79.40888,55,4.0,1,0,0,0,1,4,3,4,14,30,838.800000
4,VXH7zXcZzXlmAVN8GSjGRQ,Thai Express,43.77488,-79.49462,5,3.0,1,0,0,0,0,1,0,2,1,1,341.400000


### End date

In [79]:
date = reviews[["business_id","date"]]

last_date = []
year = []
for d in date['date']:
    date_split = d.split("-")
    epoch = int(datetime(int(date_split[0]), int(date_split[1]), int(date_split[2]), 0, 0, 0).strftime('%s'))
    
    year.append(int(date_split[0]))
    last_date.append(epoch)

date['last_year'] = year
date['last_date'] = last_date


date.head()

/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,business_id,date,last_year,last_date
0,9_CGhHMz8698M9-PkVf0CQ,2012-05-11,2012,1336690800
1,5r6-G9C4YLbC7Ziz57l3rQ,2013-02-09,2013,1360368000
2,z8oIoCT1cXz7gZP5GeU5OA,2013-05-01,2013,1367362800
3,XWTPNfskXoUL-Lf32wSk0Q,2011-09-28,2011,1317164400
4,RtUvSWO_UZ8V3Wpj0n077w,2012-12-03,2012,1354492800


In [83]:
date = date.groupby(["business_id"],as_index=False).max()
date.drop(labels=['date'],axis=1,inplace=True)
question2_1 = pd.merge(question2_1,date)
question2_1.head()

,business_id,name,latitude,longitude,review_count,stars,is_open,Italian,Japanese,Pizza,Coffee & Tea,stars_1,stars_2,stars_3,stars_4,stars_5,length,last_year,last_date
0,l09JfMeQ6ynYs5MCJtrcmQ,Alize Catering,43.71140,-79.39934,12,3.0,0,1,0,0,0,2,2,4,3,1,1475.750000,2012,1334444400
1,1K4qrnfyzKzGgJPBEcJaNQ,Chula Taberna Mexicana,43.66926,-79.33590,39,3.5,1,0,0,0,0,0,4,11,20,4,643.153846,2017,1511827200
2,nbhBRhZtdaZmMMeb2i02pg,Sunnyside Grill,43.78182,-79.49043,3,5.0,1,0,0,0,0,0,0,0,0,3,218.333333,2017,1512777600
3,FXHfcFVEfI1vVngW2gVOpw,Bampot House of Tea & Board Games,43.66158,-79.40888,55,4.0,1,0,0,0,1,4,3,4,14,30,838.800000,2017,1510099200
4,VXH7zXcZzXlmAVN8GSjGRQ,Thai Express,43.77488,-79.49462,5,3.0,1,0,0,0,0,1,0,2,1,1,341.400000,2017,1512172800


In [85]:
question2_1.to_csv("data/question2_1.csv",index=False)

### NLP approach